In [ ]:
import torch
from helpers.memory import check_memory
from transformers import AutoTokenizer, AutoModelForCausalLM
from collections import defaultdict
from helpers.expert_specialization import get_context_aware_test_data, get_js_distance

In [ ]:
hf_model_id = 'Qwen/Qwen1.5-MoE-A2.7B-Chat'

tokenizer = AutoTokenizer.from_pretrained(hf_model_id, add_eos_token = False, add_bos_token = False, padding_side = 'left')
model = AutoModelForCausalLM.from_pretrained(hf_model_id, torch_dtype = torch.bfloat16,trust_remote_code = True).cuda()

check_memory()

In [ ]:
# Test forward pass

# For Qwen2 MoE, just use output_router_logits.
# No need for hooks! https://github.com/huggingface/transformers/blob/main/src/transformers/models/qwen2_moe/modeling_qwen2_moe.py
inputs = tokenizer(['Hi this is dog', 'Where is the beef'], return_tensors = 'pt', padding = 'max_length', truncation = True, max_length = 512).to(model.device)

with torch.no_grad():
    outputs = model(**inputs, output_router_logits = True)

# Now parse the router logits
# Suppose the model has L layers with MoE. outputs.router_logits is a tuple of length L.
all_topk_experts = ()
for l, layer_router_logits in enumerate(outputs.router_logits):
    # layer_router_logits is shape [B*N, num_experts]
    gating_probs = torch.softmax(layer_router_logits, dim = -1)
    _, topk_experts = torch.topk(gating_probs, k = model.config.num_experts_per_tok, dim = -1)
    all_topk_experts += (topk_experts,) 


In [ ]:
@torch.no_grad()
def get_context_awareness_metric(model, test_token_data_list):
    """
    Compute the CA metric for each test token x layer, using JS distance to compare each meaning-specific expert distribution vs. 
     the overall distribution for that token.
    
    Params:
        @model: The model, must return `all_topk_experts` which is a tuple of length equal to # layers, where each element of
          the tuple is a BN x topk tensor of selected expert IDs.
        @test_token_data_list: A list of dictionaries of the exact format returned by `get_context_aware_test_data`.

    Returns:
        A dict of format:
            {
                test_token1: {0: .52, 1: .34, ...},
                test_token2: {0: .55, 1: .62, ...},
                ...
            },
          where the keys represent layer indices and the values represent context-awareness scores between 0 - 1
    """
    results = {}
    model.eval()

    # Number of layers from the model config
    n_layers = model.config.num_hidden_layers

    for test_item in test_token_data_list:
        test_token = test_item["test_token"]
        test_token_id = test_item["test_token_id"]
        test_meanings = test_item["test_meanings"]
        dl = test_item["dl"]

        meaning_counts_per_layer = [
            [defaultdict(int) for _ in range(len(test_meanings))]  # one dict per meaning
            for _ in range(n_layers)
        ] # meaning_counts_per_layer[l][meaning_idx][expert_id] -> count per meaning
        total_counts_per_layer = [defaultdict(int) for _ in range(n_layers)] # total_counts_per_layer[l][expert_id]: count for the all meanings baseline

        # Map each meaning string to an index
        meaning_to_idx = {m: i for i, m in enumerate(test_meanings)}

        for batch in dl:
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            batch_meanings = batch["test_meanings"]
            B, N = input_ids.shape

            outputs = model(input_ids, attention_mask, output_router_logits = True)
            all_topk_experts = ()
            for l, layer_router_logits in enumerate(outputs.router_logits):
                # layer_router_logits is shape [B*N, num_experts]
                gating_probs = torch.softmax(layer_router_logits, dim = -1)
                _, topk_experts = torch.topk(gating_probs, k = model.config.num_experts_per_tok, dim = -1)
                all_topk_experts += (topk_experts,) 

            # Flatten the input IDs for indexing alignment with all_topk_experts
            flat_input_ids = input_ids.view(-1)  # shape (B*N, )
            
            for l in range(n_layers):
                layer_experts = all_topk_experts[l]  # shape (B*N, topk)

                for token_index in range(B * N):
                    if flat_input_ids[token_index].item() == test_token_id:
                        b_idx = token_index // N # Figure out which example in the batch we belong to
                        meaning_label = batch_meanings[b_idx]
                        meaning_idx = meaning_to_idx[meaning_label]

                        # Gather all top-k experts
                        topk_exs = layer_experts[token_index]  # shape (topk,)
                        for ex_val in topk_exs:
                            ex_id = int(ex_val.item())
                            meaning_counts_per_layer[l][meaning_idx][ex_id] += 1
                            total_counts_per_layer[l][ex_id] += 1


        # Now compute the average JS distance for each layer (comparing each meaning vs. the overall distribution) then averaging
        layer_js_distances = []

        for l in range(n_layers):
            layer_sense_dists = []
            for s_idx in range(len(test_meanings)):
                d_js = get_js_distance(meaning_counts_per_layer[l][s_idx], total_counts_per_layer[l])
                layer_sense_dists.append(d_js)

            if len(layer_sense_dists) > 0:
                avg_js = sum(layer_sense_dists) / len(layer_sense_dists)
            else:
                avg_js = 0.0

            layer_js_distances.append(avg_js)

        results[test_token] = {i: val for i, val in enumerate(layer_js_distances)}

    return results

@torch.no_grad()
def get_token_specialization_metric(model, test_token_data_list, pad_token_id):
    """
    Computes a token specialization metric for each test token x layer, using the JS distance b/t: (a) the distribution of 
      experts used for that token and (b) the distribution of experts used for *all* tokens (excluding padding).
    
    Params:
        @model: The model, must return `all_topk_experts` which is a tuple of length equal to # layers, where each element of
          the tuple is a BN x topk tensor of selected expert IDs.
        @test_token_data_list: A list of dictionaries of the exact format returned by `get_context_aware_test_data`.
        @pad_token_id: The ID used for padding, which we should exclude from the "global usage" distribution.

    Returns:
        A dict of format:
            {
                test_token1: {0: .52, 1: .34, ...},
                test_token2: {0: .55, 1: .62, ...},
                ...
            },
          where the keys represent layer indices and the values represent token-specialization scores between 0 - 1
    """

    model.eval()
    n_layers = model.config.num_hidden_layers

    results = {}

    for token_item in test_token_data_list:
        token_str = token_item["test_token"]
        token_id = token_item["test_token_id"]
        dl = token_item["dl"]

        # For each layer, we'll track:
        token_expert_counts = [defaultdict(int) for _ in range(n_layers)] # token_expert_counts[l][expert_id] = # of times `token_id` is assigned to expert_id
        global_expert_counts = [defaultdict(int) for _ in range(n_layers)] # global_expert_counts[l][expert_id] = # of times ANY non-pad token is assigned to expert_id
 
        for batch in dl:
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            B, N = input_ids.shape

            outputs = model(input_ids, attention_mask, output_router_logits = True)
            all_topk_experts = ()
            for l, layer_router_logits in enumerate(outputs.router_logits):
                # layer_router_logits is shape [B*N, num_experts]
                gating_probs = torch.softmax(layer_router_logits, dim = -1)
                _, topk_experts = torch.topk(gating_probs, k = model.config.num_experts_per_tok, dim = -1)
                all_topk_experts += (topk_experts,) 

            flat_ids = input_ids.view(-1)  # shape B*N
            # for each layer, shape (B*N, topk)
            for l in range(n_layers):
                layer_experts = all_topk_experts[l]  # (B*N, topk)
                for idx in range(B*N):
                    # skip if it's padded
                    if flat_ids[idx].item() == pad_token_id:
                        continue

                    # Add global usage counts
                    topk_exs = layer_experts[idx]
                    for ex_id_val in topk_exs:
                        ex_id = int(ex_id_val.item())
                        global_expert_counts[l][ex_id] += 1

                    # If it's our target token, also track token_expert_counts
                    if flat_ids[idx].item() == token_id:
                        for ex_id_val in topk_exs:
                            ex_id = int(ex_id_val.item())
                            token_expert_counts[l][ex_id] += 1

        # Now compute the JS distance for each layer, comparing token_expert_counts vs. global_expert_counts
        layer_js_list = []
        for l in range(n_layers):
            d_js = get_js_distance(token_expert_counts[l], global_expert_counts[l])
            layer_js_list.append(d_js)

        results[token_str] = {i: val for i, val in enumerate(layer_js_list)}

    return results

In [ ]:
context_aware_test_dataset = get_context_aware_test_data("./../../data/contextual-tokens/samples_*.yaml", tokenizer, 512, 128)

In [ ]:
context_awareness = get_context_awareness_metric(model, context_aware_test_dataset)
token_specialization = get_token_specialization_metric(model, context_aware_test_dataset)